In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image

In [3]:
tf.config.list_physical_devices('GPU')

2023-02-11 11:17:11.487165: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


[]

Load the MNIST dataset


In [2]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

X_valid, X_train = X_train_full[:10000] / 255.0, X_train_full[10000:] / 255.0
y_valid, y_train = y_train_full[:10000], y_train_full[10000:]
X_test = X_test / 255.0

In [3]:
# reshape is needed to add the 4th dimension that represents the channel (grayscale)
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)

# Alternatively, the solutions notebook does it this way
# X_train = X_train[..., np.newaxis]
# X_valid = X_valid[..., np.newaxis]
# X_test = X_test[..., np.newaxis]


Build the CNN

In [4]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu", padding="same",
                        input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"),
    keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation="softmax")
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         2

2023-02-07 20:02:33.972328: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-07 20:02:33.972565: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 20:02:33.973779: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=keras.metrics.sparse_categorical_accuracy)

Train the model

In [7]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

2023-02-07 20:03:12.883129: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-02-07 20:03:12.906632: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2099940000 Hz


Epoch 1/30
1563/1563 [==============================] - 537s 343ms/step - loss: 1.2722 - sparse_categorical_accuracy: 0.5315 - val_loss: 0.0929 - val_sparse_categorical_accuracy: 0.9756
Epoch 2/30
1563/1563 [==============================] - 432s 276ms/step - loss: 0.1545 - sparse_categorical_accuracy: 0.9615 - val_loss: 0.0762 - val_sparse_categorical_accuracy: 0.9818
Epoch 3/30
1563/1563 [==============================] - 434s 278ms/step - loss: 0.1043 - sparse_categorical_accuracy: 0.9733 - val_loss: 0.0517 - val_sparse_categorical_accuracy: 0.9871
Epoch 4/30
1563/1563 [==============================] - 444s 284ms/step - loss: 0.0815 - sparse_categorical_accuracy: 0.9805 - val_loss: 0.0722 - val_sparse_categorical_accuracy: 0.9839
Epoch 5/30
1563/1563 [==============================] - 447s 286ms/step - loss: 0.0811 - sparse_categorical_accuracy: 0.9801 - val_loss: 0.0517 - val_sparse_categorical_accuracy: 0.9875
Epoch 6/30
1563/1563 [==============================] - 448s 286ms/ste

KeyboardInterrupt: 

In [ ]:
X_test = X_test.reshape(-1, 28, 28, 1)
model.evaluate(X_test, y_test)

313/313 [==============================] - 43s 139ms/step - loss: 0.0341 - sparse_categorical_accuracy: 0.9921


[0.03408491611480713, 0.9921000003814697]